In [1]:
# ler todos os arquivos csv do diretório e guardar em um objeto
# import required modules
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd
from collections import Counter
from tqdm import tqdm
diretorio = r'Bases/Arquivos_Limpos/'
all_files = list(filter(lambda x: '.csv' in x, os.listdir(diretorio)))

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Layout CNPJ
CNPJ = {"EMPRE" : ['CNPJ_BASE', 'RAZAO_SOCIAL', 'NATUREZA_JURIDICA' , 'QUALIFICACAO' ,
          'VALOR_CAPITAL_SOCIAL' , 'PORTE_EMPRESA'  , 'ENTE_FEDERATIVO'],

"ESTABELE" : ['CNPJ_BASE', 'CNPJ_ORDEM' , 'CNPJ_DV' , 'MATRIZ_FILIAL'  , 'NOME_FANTASIA' , 'SITUACAO_CADASTRAL' ,
              'DATA_SITUACAO_CADASTRAL'  , 'MOTIVO_SITUACAO_CADASTRAL'  , 'DATA_INICIO_ATIVIDADE'  ,
              'CNAE_PRINCIPAL'  , 'CNAE_SECUNDARIO' , 'TIPO_LOGRADOURO'  , 'LOGRADOURO'  , 'NUMERO'  , 'COMPLEMENTO' ,
              'BAIRRO'  , 'CEP'  , 'UF'  , 'MUNICIPIO'  , 'DDD1'  , 'TELEFONE1'  , 'DDD2'  , 'TELEFONE2'  ,
              'DDD_FAX'  , 'FAX'  , 'EMAIL'],  

"SIMPLES" : ['CNPJ_BASE'  , 'OPCAO_SIMPLES'  , 'DATA_OPCAO_SIMPLES'  , 'DATA_EXCLUSAO_SIMPLES'  ,
             'OPCAO_MEI'  , 'DATA_OPCAO_MEI'  , 'DATA_EXCLUSAO_MEI'],

"SOCIO" : ['CNPJ_BASE'  , 'TIPO'  , 'NOME'  , 'CPF_CNPJ'  , 'QUALIFICACAO'  , 'DATA_ENTRADA' ,
            'PAIS' , 'REPRESENTANTE'  , 'NOME_REPRESENTANTE'  , 'QUALIFICACAO_REPRESENTANTE'  , 'FAIXA_ETARIA'  ],

"PAIS" : ['CODIGO_PAIS' , 'PAIS'],

"MUNIC" : ['MUNICIPIO'  , 'MUNICIPIO'  ],

"QUALS" : ['QUALIFICACAO', 'QUALIFICACAO' ],

"NATJU" : ['NATUREZA_JURIDICA'  , 'NATUREZA_JURIDICA'],

"MOTI" : ['MOTIVO_SITUACAO_CADASTRAL'  , 'MOTIVO_SITUACAO_CADASTRAL'  ],

"CNAE" : ['CODIGO_CNAE'  , 'CNAE' ]}


In [3]:
cartoes = pd.read_excel(r"C:\Users\ABRASEL NACIONAL\Documents\CNPJ_PROGRAMATICA\Painel_Leads\BaseCsv\ATUALIZADO_December_Estabelecimentos_BenVisaVale.xlsx" , usecols=[0,1,3])
cartoes.head(5)


,Estabelecimento,Endereço,Telefone
0,GRAN PASTEL GOURMET,"RUA CARACA 934, 934 CASA NOSSA SENHORA DA CONC...",(31) 3222-3020
1,M130 SERRA,"R IVAI, 130 SERRA, BELO HORIZONTE - MG",(31) 2555-3395
2,COMBATE DISTRIBUIDORA,"RUA BAMBUI 56, 56 LOJA A NOSSA SENHORA DA CONC...",(31) 3444-0680
3,RESTAURANTE GFH 8 EIRELI,"RUA ORIENTE 827, 2323 PORTARIA - SERRA, BELO H...",(31) 3273-4341
4,EPA SUPERMERCADOS,"R CARACA, 600 SERRA, BELO HORIZONTE - MG",Indisponível


In [4]:
dtypes = {'CNPJ_BASE': 'object',
 'CNPJ_ORDEM': 'object',
 'CNPJ_DV': 'object'}
df_1 = pd.read_csv(f'{diretorio}{all_files[0]}', names=['CNPJ_BASE', 'CNPJ_ORDEM' , 'CNPJ_DV','NOME_FANTASIA','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO'], sep=';',encoding='utf-8', usecols=[0,1,2,4, 12, 13, 14, 15], low_memory=False, dtype=dtypes)
print(df_1.columns)
df_1['CNPJ'] = df_1['CNPJ_BASE'].map(str) + df_1['CNPJ_ORDEM'].map(str) + df_1['CNPJ_DV'].map(str)
df_1['LOGRADOURO_NUMERO'] = df_1['LOGRADOURO'].map(str) + ' ' + df_1['NUMERO'].map(str)
df_1['LOGRADOURO_NUMERO'].replace(regex='nan', value='', inplace=True)
df_1['COMPLEMENTO_BAIRRO'] = df_1['COMPLEMENTO'].map(str) + ' ' + df_1['BAIRRO'].map(str)
df_1['COMPLEMENTO_BAIRRO'].replace(regex='nan', value='', inplace=True)
df_1.drop(['CNPJ_BASE', 'CNPJ_ORDEM' , 'CNPJ_DV','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO'],axis=1, inplace=True)
df_1.head(5)


Index(['CNPJ_BASE', 'CNPJ_ORDEM', 'CNPJ_DV', 'NOME_FANTASIA', 'LOGRADOURO',
       'NUMERO', 'COMPLEMENTO', 'BAIRRO'],
      dtype='object')


,NOME_FANTASIA,CNPJ,LOGRADOURO_NUMERO,COMPLEMENTO_BAIRRO
0,AGO RESTOBAR,39228727000166,CLOUTIDES PEREIRA ROCHA SN,BAIRRO CEARENSE
1,Indisponível,39228909000137,GETULIO VARGAS 158,CENTRO
2,PROMESSA DE DEUS,39232903000133,SALAO 681,BLOCO CASA RUA AUGUSTINHO ALEGRE CHIC
3,PESQUE & PAGUE BOM PESQUEIRO,39233197000144,RODOVIA PR364 SN,CABECEIRA DO ARROIO GRANDE
4,BAR DO BEM,39237626000151,PARA 440,PACOVAL


In [ ]:
NOMEBEN_OK = []
for row in tqdm(range(0,len(df_1['NOME_FANTASIA']))):
    if df_1.at(row,'NOME_FANTASIA') in lista_lojas_cartoes:
        NOMEBEN_OK.append(1)
    else:
        NOMEBEN_OK.append(0)
print(Counter(NOMEBEN_OK).keys())
print(Counter(NOMEBEN_OK).values())